In [69]:
#in the name  of God

#import libraries block
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [70]:
#constant definitions
bestTrainAccuracy = 0
bestValueAccuracy = 0
goalLearningRate = 0
bestReg = 0
learningRatesList = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
regList= [0.001 ,0.01 ,0.1, 0.5, 0.75, 1, 5, 10, 50]

In [71]:
#functions definitions block

#calculate number of missClassified Data 
def NumberOfMissClassifiedData(y, p) :
    sum = 0
    for i in range(len(y)) :
        if y[i] != p[i] :
            sum+=1
    return sum 

#logistic model definition
def model(rate, reg, maxIteration, tolerance, X, Y) :
    X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
    W = np.zeros(X.shape[1])
    
    #in this loop we train the model by checking tolerance
    for i in range(maxIteration) :
        err = (1 / (1 + np.exp(-(X @ W)))) - Y
        gradian_delta = (((1 / reg) * (np.transpose(X) @ err)) + np.sum(W))
        if np.all(abs(gradian_delta) >= tolerance):
            W -= rate * gradian_delta / (X.shape[1])
        else:
            break
    
    return W

In [72]:
#reading test and train files
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

#now we display number of missing data in each columns of train data and in test data
for i in list(train_data.columns) :
    print(i, ":", sum(train_data[i].isnull()), end=" - ")
print("\n\n\n")
for i in list(test_data.columns) :
    print(i, " : ", sum(test_data[i].isnull()), end=" - ")


PassengerId : 0 - Survived : 0 - Pclass : 0 - Name : 0 - Sex : 0 - Age : 177 - SibSp : 0 - Parch : 0 - Ticket : 0 - Fare : 0 - Cabin : 687 - Embarked : 2 - 



PassengerId  :  0 - Pclass  :  0 - Name  :  0 - Sex  :  0 - Age  :  86 - SibSp  :  0 - Parch  :  0 - Ticket  :  0 - Fare  :  1 - Cabin  :  327 - Embarked  :  0 - 

In [73]:
#in this section we replace missing datas and convert non numerical features to numerical features
#and convert to numrical features and replace them empty places

#fill missing data of age column with median of each sex of each pclass for test and train data
train_data['Age'] = train_data.groupby(['Sex', 'Pclass'])['Age'].apply(lambda i: i.fillna(i.median()))
test_data['Age'] = test_data.groupby(['Sex', 'Pclass'])['Age'].apply(lambda i: i.fillna(i.median()))

#convert non numerical features to numerical features
#convert values of Sex features to 0(male),1(female)
#convert values of Embarked features to 0(C),1(Q),2(S)
train_data['Sex'].replace(["male", "female"], [0.0,1.0], inplace=True)
test_data['Sex'].replace(["male", "female"], [0.0,1.0], inplace=True)
train_data["Embarked"].replace(["C", "Q", "S"], [0.0, 1.0, 2.0], inplace=True)
test_data["Embarked"].replace(["C", "Q", "S"], [0.0, 1.0, 2.0], inplace=True)

#we delete Inappropriate features from train and test data
train_data = train_data.drop(["Cabin", "Name", "PassengerId", "Ticket"], axis=1)
test_data = test_data.drop(["Cabin", "Name", "PassengerId", "Ticket"], axis=1)

#then we replace missing values by median of column
train_data["Embarked"] = train_data["Embarked"].fillna(train_data["Embarked"].median())
test_data["Fare"] = test_data["Fare"].fillna(test_data["Fare"].median())

#initialize X_text, X_train, Y_train and print its shape
X_train = train_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
Y_train = train_data[["Survived"]]
X_test = test_data.to_numpy()
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()
print(X_test.shape)
print(X_train.shape)
print(Y_train.shape)

(418, 7)
(891, 7)
(891, 1)


In [74]:
#first normalize X_train and X_test
#then print them shapes and five first elements
X_train = (X_train - np.mean(X_train, axis=0)) / (np.sqrt(np.var(X_train, axis=0)))
print(X_train.shape)
print(X_train[:5])

X_test = (X_test - np.mean(X_test, axis=0)) / (np.sqrt(np.var(X_test, axis=0)))
print(X_test.shape)
print(X_test[:5])

(891, 7)
[[ 0.82737724 -0.73769513 -0.53489116  0.43279337 -0.47367361 -0.50244517
   0.58595414]
 [-1.56610693  1.35557354  0.66839176  0.43279337 -0.47367361  0.78684529
  -1.9423032 ]
 [ 0.82737724  1.35557354 -0.23407043 -0.4745452  -0.47367361 -0.48885426
   0.58595414]
 [-1.56610693  1.35557354  0.44277621  0.43279337 -0.47367361  0.42073024
   0.58595414]
 [ 0.82737724 -0.73769513  0.44277621 -0.4745452  -0.47367361 -0.48633742
   0.58595414]]
(418, 7)
[[ 0.87348191 -0.75592895  0.39945123 -0.49947002 -0.4002477  -0.49741333
  -0.47091535]
 [ 0.87348191  1.32287566  1.35927311  0.61699237 -0.4002477  -0.51227801
   0.70076689]
 [-0.31581919 -0.75592895  2.51105936 -0.49947002 -0.4002477  -0.46410047
  -0.47091535]
 [ 0.87348191 -0.75592895 -0.1764419  -0.49947002 -0.4002477  -0.48247516
   0.70076689]
 [ 0.87348191  1.32287566 -0.56037065  0.61699237  0.61989583 -0.4174915
   0.70076689]]


In [75]:
#main
train_data = train_data.to_numpy()
for rate in learningRatesList :
    for reg in regList :
        #MCV : miss Classififed Values List
        #MCL : miss Classififed Train List
        MCV = []
        MCL = []
        for i in range(10) :
            np.random.shuffle(train_data)
            X_train = train_data[: , 1 : ]
            Y_train = train_data[: , 0]
            X_train = (X_train - np.mean(X_train, axis=0)) / (np.sqrt(np.var(X_train, axis=0)))
            #because 5 fold cross validation
            N = X_train.shape[0]//5
            for i in range(5) :
                xt = np.concatenate((X_train[ : i * N], X_train[(i+1) * N : ]), axis=0)
                xv = X_train[i * N : (i + 1) * N] 
                yt = np.concatenate((Y_train[ : i * N], Y_train[(i+1) * N : ]), axis=0)
                yv = Y_train[i * N : (i + 1) * N]
                weights = model(rate, reg, 500, 0.001, xt, yt)
                #calculateing missClassified value and missClassified List
                MCL.append(NumberOfMissClassifiedData(yt, np.round(1 / (1 + np.exp(-((xt @ weights[1:]) + weights[0]))))))
                MCV.append(NumberOfMissClassifiedData(yv, np.round(1 / (1 + np.exp(-((xv @ weights[1:]) + weights[0]))))))

        print("learning rate = ", rate , "regularization constant = ", reg)
        #calculate train and value Accuraicy
        trainAcc =100.0 - ((sum(MCL)/len(MCL))/713)*100
        valueAcc =100.0 - ((sum(MCV)/len(MCV))/178)*100
        print("accuarcy in train: ", trainAcc, "accuarcy in validation data: ", valueAcc, "\n\n")
        if trainAcc > bestTrainAccuracy and valueAcc > bestValueAccuracy :
            bestTrainAccuracy = trainAcc
            bestValueAccuracy = valueAcc
            goalLearningRate = rate
            bestReg = reg

c:\Users\Lenovo\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
c:\Users\Lenovo\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: overflow encountered in exp
c:\Users\Lenovo\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in exp


learning rate =  0.1 regularization constant =  0.001
accuarcy in train:  72.28330995792426 accuarcy in validation data:  72.53932584269663 


learning rate =  0.1 regularization constant =  0.01
accuarcy in train:  71.89621318373071 accuarcy in validation data:  71.56179775280899 


learning rate =  0.1 regularization constant =  0.1
accuarcy in train:  72.8835904628331 accuarcy in validation data:  72.21348314606742 


learning rate =  0.1 regularization constant =  0.5
accuarcy in train:  74.49649368863956 accuarcy in validation data:  73.92134831460675 


learning rate =  0.1 regularization constant =  0.75
accuarcy in train:  75.50070126227209 accuarcy in validation data:  74.97752808988764 


learning rate =  0.1 regularization constant =  1
accuarcy in train:  76.44319775596072 accuarcy in validation data:  76.11235955056179 


learning rate =  0.1 regularization constant =  5
accuarcy in train:  80.08415147265077 accuarcy in validation data:  79.58426966292134 


learning rate 

In [76]:
print("best accuarcy on train : ", bestTrainAccuracy)
print("best accuarcy on validation : ", bestValueAccuracy)
print("learning rate : ", goalLearningRate)
print("regularization constant : ", bestReg)

#use from model
weights = model(goalLearningRate, bestReg, 500, 0.001, X_train, Y_train)
#print predicate y
print(np.round(1 / (1 + np.exp(-((X_test @ weights[1:]) + weights[0])))))

best accuarcy on train :  80.21598877980364
best accuarcy on validation :  79.71910112359551
learning rate :  0.3
regularization constant :  5
[0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1.
 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0.
 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0.